## Utils

In this section, the basic functions required to run the codes are defined.

In [1]:
import copy
import json
import sys, os
import time

from tqdm import tqdm as h_tqdm


# data_dir = './data'

def check_path(home):
    if not os.path.isdir(home):
        print(f'Not found dir: {home}, creating it.')
        os.mkdir(home)

def red_str(s, tofile=False):
    s = str(s)
    if tofile:
        # s = f'**{s}**'
        pass
    else:
        s = f'\033[1;31;40m{s}\033[0m'
    return s


def get_time_str():
    return time.strftime('%Y-%m-%d_%H.%M.%S') + str(time.time() % 1)[1:6]


def save_json(data, fn, sort_keys=True, indent=None):
    with open(fn, 'w') as f:
        json.dump(data, f, sort_keys=sort_keys, indent=indent)


def load_json(fn):
    with open(fn, 'r') as f:
        return json.load(f)


def gen_data_from_jsonl(fn):
    with open(fn, 'r') as f:
        for line in f:
            data = json.loads(line)
            yield data


class Logger:
    def __init__(self, fn, verbose=1):
        self.pre_time = time.time()
        self.fn = fn
        self.verbose = verbose

    def __str__(self):
        return self.fn

    def log(self, s='', end='\n', red=False):
        s = str(s)
        if self.verbose == 1:
            p = red_str(s) if red else s
            print(p, end=end)
        elif self.verbose == 2:
            p = red_str(s, tofile=True) if red else s
            print(p, end=end)
        now_time = time.time()
        s = s + end
        if now_time - self.pre_time > 30 * 60:
            s = get_time_str() + '\n' + s
            self.pre_time = now_time
        with open(self.fn, 'a') as f:
            fs = red_str(s, tofile=True) if red else s
            f.write(fs)
        sys.stdout.flush()


class Graph:
    def __init__(self, min_cnt=1):
        self.min_cnt = min_cnt
        self.edge_cnt = {}
        self.adj = {}
        self._nb_edges = 0

    def add_edge(self, a, b):
        e = (a, b)
        self.edge_cnt.setdefault(e, 0)
        self.edge_cnt[e] += 1

        if self.edge_cnt[e] == self.min_cnt:
            self.adj.setdefault(a, [])
            self.adj[a].append(b)
            self._nb_edges += 1

    def has_edge(self, a, b):
        cnt = self.edge_cnt.get((a, b), 0)
        return cnt >= self.min_cnt

    def get_edges(self):
        edges = sorted([(a, b) for (a, b), cnt in self.edge_cnt.items() if cnt >= self.min_cnt])
        return edges

    def get_adj(self, a):
        return self.adj.get(a, [])

    def nb_edges(self):
        return self._nb_edges


# noinspection PyUnresolvedReferences
class Object(object):
    def __init__(self, **kwargs):
        for name in kwargs:
            setattr(self, name, kwargs[name])

    def vars(self):
        return self.__dict__

    def keys(self):
        return sorted(self.__dict__.keys())

    def values(self, keys=None):
        if keys is None:
            keys = self.keys()
        return [self.__dict__[k] for k in keys]

    def copy(self):
        return copy.deepcopy(self)

    def get(self, name, d=None):
        return self.__dict__.get(name, d)

    def set(self, name, value):
        self.__dict__[name] = value

    def filter(self, *keys):
        ret = Object()
        for k in keys:
            ret.set(k, self.get(k))
        return ret

    def update(self, **args):
        self.__dict__.update(args)
        return self

    def setdefault(self, **args):
        for k, v in args.items():
            if k not in self.__dict__:
                self.__dict__[k] = v
        return self

    def prt_json(self):
        d = {}
        for k, v in self.__dict__.items():
            if type(v) not in (dict, list, int, float, bool, str):
                v = str(v)
            d[k] = v
        return json.dumps(d, indent=2, sort_keys=True)

    def prt_line(self):
        return json.dumps(self.__dict__, sort_keys=True)

    def __str__(self):
        lines = []
        for k in sorted(self.keys()):
            lines.append(f'{k}: {self.get(k)}')
        return '\n'.join(lines)

    def __repr__(self):
        return self.__str__()

    def format_prt(self):
        type_name = type(self).__name__
        arg_strings = []
        star_args = {}
        for arg in self._get_args():
            arg_strings.append(repr(arg))
        for name, value in self._get_kwargs():
            if name.isidentifier():
                arg_strings.append('%s=%r' % (name, value))
            else:
                star_args[name] = value
        if star_args:
            arg_strings.append('**%s' % repr(star_args))
        return '%s(%s)' % (type_name, ', '.join(arg_strings))

class AsyncStream:
    def __init__(self, stream, max_q_size=1, mode='MT'):
        self.stream = stream
        self.close = False
        if mode == 'MP':
            from multiprocessing import Process, Queue
            self.q = Queue(max_q_size)
            self.p = Process(target=self.task)
            self.p.daemon = True
            self.p.start()
        else:
            import threading
            from queue import Queue
            self.q = Queue(max_q_size)
            self.p = threading.Thread(target=self.task)
            self.p.daemon = True
            self.p.start()

    def task(self):
        for d in self.stream:
            self.q.put(d)
        self.q.put(None)

    def generator(self):
        while True:
            d = self.q.get()
            if d is None:
                break
            yield d

class my_tqdm:
    def __init__(self, desc, total, leave):
        self.desc = desc
        self.total = total
        self.cnt = 0
        self.leave = leave
        if self.leave:
            print(f'>>> begin {self.desc}...')
        self.bt = time.time()

    def update(self, n):
        self.cnt += n

    def close(self):
        t = time.time() - self.bt
        v = self.cnt / t
        if self.leave:
            print(f'--- {self.desc} end, cnt: {self.cnt}, time: {t:.0f}s, v: {v:.2f}it/s')


def tqdm(verbose=None, desc='tqdm', leave=True, total=None):
    if verbose is None:
        verbose = args.get('verbose', 1)
    if verbose == 1:
        return h_tqdm(desc=desc, total=total, leave=leave, ncols=90, ascii=True)
    return my_tqdm(desc=desc, total=total,  leave=leave)

args = Object()
run_time_dir = 'run_time'
check_path(run_time_dir)

data_dir = f'{run_time_dir}/data'
log_dir = f'{run_time_dir}/log'
save_dir = f'{run_time_dir}/save'

check_path(data_dir)
check_path(log_dir)
check_path(save_dir)



def main():
    pass


if __name__ == '__main__':
    main()

### Base

In this part, the designed model is implemented. The proposed change in the thesis has been applied using the star_GNN function in the model of the reference article. In fact, by adding this function, the performance of the model has been improved and the fundamental change of the codes has been in this part.

In [2]:
import numpy as np, sys, math, os
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import utils
from utils import args

eps = 1e-7
inf = 1e31


class Base:
    deep = True
    args = Object()

    # feature: [type..], [tags..], mid
    def __init__(self, data):
        self.raw_adjs = data.adjs

        # self.save_name = f'{utils.save_dir}/{args.run_name}/model.ckpt'
        self.save_dir = f'{save_dir}/{args.run_name}'
        self.tb_name = f'{save_dir}/{args.run_name}'

        self.graph = tf.Graph()
        with self.graph.as_default():

            #tf.random.set_seed(args.seed)
            tf.set_random_seed(args.seed)
            self.compile()
        self.fit_step = 0

    def compile(self):
        self.make_io()
        self.make_model()
        if args.run_tb:
            self.all_summary = tf.summary.merge_all()
            self.tbfw = tf.summary.FileWriter(self.tb_name, self.sess.graph)

    def placeholder(self, dtype, shape, name, to_list):
        ph = tf.placeholder(dtype, shape, name)
        self.placeholder_dict[name] = ph
        to_list.append(ph)
        return ph

    def make_io(self):
        self.placeholder_dict = {}
        self.inputs = []
        L = args.seq_length
        self.placeholder(tf.int32, [None, L], 'share_seq', self.inputs)
        self.placeholder(tf.int32, [None, L], 'click_seq', self.inputs)

        self.placeholder(tf.int32, [None], 'pos', self.inputs)
        self.placeholder(tf.int32, [None, None], 'neg', self.inputs)

        self.adjs = [
            tf.constant(adj, dtype=tf.int32) for adj in self.raw_adjs
        ]  # [N, M] * 4, in_0, out_0, in_1, out_1

    def get_data_map(self, data):
        data_map = dict(zip(self.inputs, data))
        return data_map

    def make_model(self):
        with tf.variable_scope(
                'Graph', reuse=tf.AUTO_REUSE,
                regularizer=self.l2_loss('all')) as self.graph_scope:
            n = args.nb_nodes
            k = args.dim_k
            self.embedding_matrix = tf.get_variable(name='emb_w', shape=[n, k])
            with tf.variable_scope(
                    'graph_agg', reuse=tf.AUTO_REUSE) as self.graph_agg_scope:
                pass

        with tf.variable_scope('Network',
                               reuse=tf.AUTO_REUSE,
                               regularizer=self.l2_loss('all')):
            score, label = self.forward(*self.inputs)
            #seq_loss = tf.losses.softmax_cross_entropy(label, score)
            seq_loss = tf.losses.mean_squared_error(label, score)
            tf.summary.scalar('seq_loss', seq_loss)

        self.loss = seq_loss
        self.loss += tf.losses.get_regularization_loss()

        opt = tf.train.AdamOptimizer(learning_rate=args.lr)
        self.minimizer = opt.minimize(self.loss)
        tf.summary.scalar('loss', self.loss)

        graph_var_list = tf.trainable_variables(scope='^Graph/')
        network_var_list = tf.trainable_variables(scope='^Network/')
        for v in graph_var_list:
            print('graph', v)
        for v in network_var_list:
            print('network', v)

        self.saver = tf.train.Saver()
        self.sess = self.get_session()
        self.sess.run(tf.global_variables_initializer())

    def get_session(self):
        gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=1,
            visible_device_list=args.gpu,
            allow_growth=True,
        )
        config = tf.ConfigProto(gpu_options=gpu_options)
        session = tf.Session(config=config)
        return session

    def fit(self, data):
        data = dict(zip(self.inputs, data))
        if args.run_tb:
            _, loss, summary = self.sess.run(
                [self.minimizer, self.loss, self.all_summary], data)
            self.tbfw.add_summary(summary, self.fit_step)
        else:
            _, loss = self.sess.run([self.minimizer, self.loss], data)
        self.fit_step += 1
        return loss

    def topk(self, data):
        data = self.get_data_map(data)
        return self.sess.run([self.topkV, self.topkI], data)

    def save(self):
        name = f'{self.save_dir}/model.ckpt'
        self.saver.save(self.sess, name)

    def restore(self):
        name = f'{self.save_dir}/model.ckpt'
        try:
            self.saver.restore(self.sess, name)
        except Exception as e:
            print(f'can not restore model: {name}')
            raise e

    def l2_loss(self, name):
        alpha = args.get(f'l2_{name}', 0)
        if alpha < 1e-7:
            return None
        return lambda x: alpha * tf.nn.l2_loss(x)

    def Mean(self, seq, seq_length=None, mask=None, name=None):
        # seq: (None, L, k), seq_length: (None, ), mask: (None, L)
        # ret: (None, k)
        if seq_length is None and mask is None:
            with tf.variable_scope('Mean'):
                return tf.reduce_sum(seq, -2)

        with tf.variable_scope('MaskMean'):
            if mask is None:
                mask = tf.sequence_mask(seq_length,
                                        maxlen=tf.shape(seq)[1],
                                        dtype=tf.float32)
            mask = tf.expand_dims(mask, -1)  # (None, L, 1)
            seq = seq * mask
            seq = tf.reduce_sum(seq, -2)  # (None, k)
            seq = seq / (tf.reduce_sum(mask, -2) + eps)
        return seq

    def MLP(self, x, fc, activation, name):
        with tf.variable_scope(f'MLP_{name}'):
            for i in range(len(fc)):
                x = tf.layers.dense(x,
                                    fc[i],
                                    activation=activation,
                                    name=f'dense_{i}')
        return x

    def gate(self, a, b, name):
        with tf.variable_scope(name):
            alpha = tf.layers.dense(tf.concat([a, b], -1),
                                    1,
                                    activation=tf.nn.sigmoid,
                                    name='gateW')
            ret = alpha * a + (1 - alpha) * b
        return ret

    def star_GNN(self, a , b , c , d , name):
        with tf.variable_scope(name):
            u = self.gate(a, b, 'merge_share_and_click_seq')
            
            z_r = self.gate(c , d , 'merge_share_and_click_seqn')
            #print(z_r.shape)
            #z = tf.layers.dense(tf.concat([
             #   tf.reshape(u, [args.dim_k, 1]),
             #   tf.reshape(z_r, [args.dim_k, 1])
            #], -1),
            #                    args.dim_k,
            #                    activation=tf.nn.relu)
            #z = tf.layers.dense(tf.concat([u,z_r], -1),
             #                   args.dim_k,
              #                  activation=tf.nn.relu)
            z = self.gate(z_r,u, 'merge')
            #print(u)
            #print(z_r)
            return z

    def Embedding(self, node, name='node', mask_zero=False):
        # node: [BS]
        with tf.variable_scope(f'Emb_{name}'):
            emb_w = self.embedding_matrix
            t = tf.gather(emb_w, node)
            if mask_zero:
                mask = tf.not_equal(node, 0)
                mask = tf.cast(mask, tf.float32)
            else:
                mask = None
        return t, mask


    def forward(self, share_seq, click_seq, pos, neg):
        pos2 = tf.expand_dims(pos, -1)
        nxt = tf.concat([pos2, neg], -1)  # [BS, M + 1]
        label = tf.concat([
            tf.ones_like(pos2, dtype=tf.int32),
            tf.zeros_like(neg, dtype=tf.int32)
        ], -1)  # [BS, M + 1]

        seq_emb = self.merge_seq(share_seq, click_seq)
        seq_emb = tf.layers.dense(seq_emb,args.dim_k,name='dense_W',use_bias=False)
        
        print('seq_emb',seq_emb.shape)
        print('embedding_matrix',self.embedding_matrix.shape)
        score = tf.matmul(seq_emb, self.embedding_matrix, transpose_b=True)
        #print('self.embedding_matrix', type(self.embedding_matrix))
        #score = tf.layers.dense(tf.concat([seq_emb,tf.transpose(self.embedding_matrix)], -1),52740,activation=tf.nn.relu)
        #print('score', score)

        topk = tf.math.top_k(score, k=500)
        self.topkV = topk.values
        self.topkI = topk.indices

        nxt_embs, _ = self.Embedding(nxt)  # [BS, M + 1, k]
        nxt_score = tf.reduce_sum(tf.expand_dims(seq_emb, 1) * nxt_embs, -1)
        return nxt_score, label

    def node_embedding(self, node):
        # node: [BS, L]
        embs, mask = self.Embedding(node, mask_zero=True)
        return embs, mask   

    def merge_seq(self, share_seq, click_seq):
        with tf.variable_scope(f'merge_seq', reuse=tf.AUTO_REUSE):
            share_seq_embs, share_mask = self.node_embedding(share_seq)
            share_emb = self.seq_embedding(share_seq_embs, share_mask, 'share')
            #print('share_emb',share_emb.shape)
            click_seq_embs, click_mask = self.node_embedding(click_seq)
            click_emb = self.seq_embedding(click_seq_embs, click_mask, 'click')
            #print('click_emb',click_emb.shape)
            
            share_emb_n, _ = self.Embedding(share_seq[:,-1], mask_zero=False)
            #print('share_emb_n',share_emb_n.shape)
            click_emb_n, _ = self.Embedding(click_seq[:,-1], mask_zero=False)
            #print('click_emb_n',click_emb_n.shape)
            
            #u = self.gate(share_emb, click_emb, 'merge_share_and_click_seq')
    
            #z_r = self.gate(share_emb_n,click_emb_n,'merge_share_and_click_seq_n')

            emb = self.star_GNN(share_emb , click_emb , share_emb_n , click_emb_n , 'user_emb')
            #print('z_r',z_r.shape)
            #print('u',u.shape)
            #emb=self.gate(z_r, u, 'merge')
            #print(emb.shape)
            return emb

    def seq_embedding(self, seq, mask, name):
        # seq: [BS, L, k]
        with tf.variable_scope(f'seq_embedding_{name}', reuse=tf.AUTO_REUSE):
            seq_emb = self.Mean(seq, mask=mask)
        return seq_emb


class GNN(Base):
    args = Base.args.copy().update()

    def node_embedding(self, node):
        # node: [BS, L]
        with tf.variable_scope(self.graph_scope):
            embs, mask = self.node_list_aggregate(node,
                                                  depth=args.gnnd,
                                                  mask_zero=True)  # [BS, L, k]
        return embs, mask

    def node_embedding2(self, node):
        # node: [BS, L]
        with tf.variable_scope(self.graph_scope):
            embs, mask = self.node_list_aggregate2(node,
                                                  depth=args.gnnd,
                                                  mask_zero=True)  # [BS, L, k]
        return embs, mask
    
    def node_list_aggregate2(self, nodes, depth, mask_zero, name='share'):
        # nodes: [BS, M]
        bs = tf.shape(nodes)[0]
        m = tf.shape(nodes)[0]
        nodes = tf.reshape(nodes, [bs * m])
        emb, mask = self.single_node_aggregate2(nodes, depth, mask_zero, name)
        # k = tf.shape(emb)[1]
        k = args.dim_k
        emb = tf.reshape(emb, [bs, m, k])
        if mask is not None:
            mask = tf.reshape(mask, [bs, m])
        return emb, mask

    def node_list_aggregate(self, nodes, depth, mask_zero, name='share'):
        # nodes: [BS, M]
        bs = tf.shape(nodes)[0]
        m = tf.shape(nodes)[1]
        nodes = tf.reshape(nodes, [bs * m])
        emb, mask = self.single_node_aggregate(nodes, depth, mask_zero, name)
        # k = tf.shape(emb)[1]
        k = args.dim_k
        emb = tf.reshape(emb, [bs, m, k])
        if mask is not None:
            mask = tf.reshape(mask, [bs, m])
        return emb, mask

    def _agg(self, cur, nxt, mask, name):
        return self.Mean(nxt, mask=mask)

    def _merge(self, cur, nxt):
        return cur + nxt

    def single_node_aggregate(self, node, depth, mask_zero, name='share'):
        # node: [BS], adj: [N, M]
        if depth <= 0:
            return self.Embedding(node, mask_zero=mask_zero)
        with tf.variable_scope(f'agg{depth}layer_{name}'):
            bs = tf.shape(node)[0]
            cur, cur_mask = self.single_node_aggregate(node, depth - 1,
                                                       mask_zero,
                                                       name)  # [BS, k]

            nxt_in_0 = tf.gather(self.adjs[0], node)  # [BS, M]
            nxt_in_0, nxt_in_0_mask = self.node_list_aggregate(nxt_in_0, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_out_0 = tf.gather(self.adjs[1], node)  # [BS, M]
            nxt_out_0, nxt_out_0_mask = self.node_list_aggregate(nxt_out_0, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_in_1 = tf.gather(self.adjs[2], node)  # [BS, M]
            nxt_in_1, nxt_in_1_mask = self.node_list_aggregate(nxt_in_1, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_out_1 = tf.gather(self.adjs[3], node)  # [BS, M]
            nxt_out_1, nxt_out_1_mask = self.node_list_aggregate(nxt_out_1, depth - 1, mask_zero, name)  # [BS, M, k]

            h = self._aggregate4(cur, nxt_in_0, nxt_in_0_mask, nxt_out_0,nxt_out_0_mask, nxt_in_1, nxt_in_1_mask,nxt_out_1, nxt_out_1_mask, name)  # [BS, k]
            return h, cur_mask
        
    def single_node_aggregate2(self, node, depth, mask_zero, name='share'):
        # node: [BS], adj: [N, M]
        if depth <= 0:
            return self.Embedding(node, mask_zero=mask_zero)
        with tf.variable_scope(f'agg{depth}layer_{name}'):
            bs = tf.shape(node)[0]
            cur, cur_mask = self.single_node_aggregate2(node, depth - 1,
                                                       mask_zero,
                                                       name)  # [BS, k]

            nxt_in_0 = tf.gather(self.adjs[0], node)  # [BS, M]
            nxt_in_0, nxt_in_0_mask = self.node_list_aggregate2(nxt_in_0, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_out_0 = tf.gather(self.adjs[1], node)  # [BS, M]
            nxt_out_0, nxt_out_0_mask = self.node_list_aggregate2(nxt_out_0, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_in_1 = tf.gather(self.adjs[2], node)  # [BS, M]
            nxt_in_1, nxt_in_1_mask = self.node_list_aggregate2(nxt_in_1, depth - 1, mask_zero, name)  # [BS, M, k]

            nxt_out_1 = tf.gather(self.adjs[3], node)  # [BS, M]
            nxt_out_1, nxt_out_1_mask = self.node_list_aggregate2(nxt_out_1, depth - 1, mask_zero, name)  # [BS, M, k]

            h = self._aggregate4(cur, nxt_in_0, nxt_in_0_mask, nxt_out_0,nxt_out_0_mask, nxt_in_1, nxt_in_1_mask,nxt_out_1, nxt_out_1_mask, name)  # [BS, k]
            return h, cur_mask

    def _aggregate4(self, cur, nxt_in_0, nxt_in_0_mask, nxt_out_0,nxt_out_0_mask, nxt_in_1, nxt_in_1_mask, nxt_out_1,nxt_out_1_mask, name):
        with tf.variable_scope(self.graph_agg_scope):
            nxt_in_0 = self._agg(cur, nxt_in_0, nxt_in_0_mask, 'agg_in')
            nxt_out_0 = self._agg(cur, nxt_out_0, nxt_out_0_mask, 'agg_out')
            nxt_in_1 = self._agg(cur, nxt_in_1, nxt_in_1_mask, 'agg_in')
            nxt_out_1 = self._agg(cur, nxt_out_1, nxt_out_1_mask, 'agg_out')

            nxt = nxt_in_0 + nxt_out_0 + nxt_in_1 + nxt_out_1
            o = self._merge(cur, nxt)
            return o

Instructions for updating:
non-resource variables are not supported in the long term


## Dataset

Using the codes of this part, we enter the data to train the model and simultaneously create the graph, the graph adjacency matrix and the feature matrix of the vertices. Due to the large amount of data used, we encountered some challenges in this section.

In [3]:
import numpy as np, sys, math, os
import json
import pickle
import copy
import utils
import re
from utils import args, tqdm
import time

data_home = 'run_time/data'
class DatasetReader:
    def __init__(self, ds):
        self.ds = ds

        if ds == 'yc64' or ds == 'test':
            self.N = 144527
            self.reader = self.yc64
            self.min_ts, self.max_ts = 1411604904, 1412017199
        elif ds == 'yc4':
            self.N = 2312432
            self.reader = self.yc4
            self.min_ts, self.max_ts = 1408507486, 1412017199
        elif ds == 'yc':
            self.N = 9249729
            self.reader = self.yc
            self.min_ts, self.max_ts = 1396292400, 1412017199

        dt = self.max_ts - self.min_ts
        dt2 = dt // 7 * 3
        self.train_ts = self.min_ts + dt2 * 2

    def yc(self, frac=1):
        pbar = tqdm(desc='read data', total=self.N)
        f = open(f'{data_home}/yc_1_{frac}/data.txt', 'r')
        for line in f:
            pbar.update(1)
            line = line[:-1]
            sid, vid_list_str = line.split()
            vid_list = []
            for vid in vid_list_str.split(','):
                vid, cls, ts = vid.split(':')
                cls = int(cls)  # cls: 0, 1, 2, ...
                ts = int(ts)
                vid_list.append([vid, cls, ts])
            yield vid_list
        f.close()
        pbar.close()

    def yc4(self):
        yield from self.yc(4)

    def yc64(self):
        yield from self.yc(64)

    def wx(self):
        pass

class DataProcess:
    def __init__(self, ds, adj_length, seq_length):
        self.ds = ds
        # self.adj_length = adj_length
        # self.seq_length = seq_length

        self.vid2node = {}
        self.vid2node['[MASK]'] = 0

        self.DR = DatasetReader(ds)
        self.G_in, self.G_out, self.train_data, self.test_data = self.build_graph(seq_length)

        rdm = np.random.RandomState(777)
        rdm.shuffle(self.train_data)

        rdm = np.random.RandomState(333)
        rdm.shuffle(self.test_data)

        args.update(nb_nodes=len(self.vid2node))
        args.update(nb_edges_0=self.G_in[0].nb_edges())
        args.update(nb_edges_1=self.G_in[1].nb_edges())

        self.adj_in_0 = self.build_adj(self.G_in[0], adj_length)
        self.adj_out_0 = self.build_adj(self.G_out[0], adj_length)
        self.adj_in_1 = self.build_adj(self.G_in[1], adj_length)
        self.adj_out_1 = self.build_adj(self.G_out[1], adj_length)

        self.adjs_tmp = [self.adj_in_0, self.adj_out_0, self.adj_in_1, self.adj_out_1]
        self.adjs = [a[0] for a in self.adjs_tmp]

    def build_graph(self, seq_length):
        test_seq = []
        G_in = [Graph() for i in range(2)]
        G_out = [Graph() for i in range(2)]
        train_data = []
        test_data = []
        for num_data, vid_list in enumerate(self.DR.reader()):

            vid_list_for_graph = [[] for i in range(2)]
            vid_list_for_train = [[] for i in range(2)]
            first_pos = [{} for i in range(2)]

            for i, (vid, typ, ts) in enumerate(vid_list):
                if vid not in self.vid2node:
                    self.vid2node[vid] = len(self.vid2node)

                for_train = False
                if ts < self.DR.train_ts:
                    for_train = True

                if for_train:
                    vid_list_for_graph[typ].append(vid)

                if typ == 0 and vid not in first_pos[0]:
                    share_history = vid_list_for_train[0]
                    if vid not in first_pos[1]:
                        click_history = vid_list_for_train[1]
                    else:
                        k = first_pos[1][vid]
                        click_history = vid_list_for_train[1][:k]

                    if len(click_history) >= 5 and len(share_history) >= 1:
                        seq_share = [share_history[-seq_length: ], click_history[-seq_length: ], vid]
                        if for_train:
                            train_data.append(seq_share)
                        else:
                            test_data.append(seq_share)

                if vid not in first_pos[typ]:
                    first_pos[typ][vid] = len(vid_list_for_train[typ])
                vid_list_for_train[typ].append(vid)

            for typ in range(2):
                for i, vid in enumerate(vid_list_for_graph[typ]):
                    if i == 0:
                        continue
                    now_node = self.vid2node[vid]
                    pre_node = self.vid2node[vid_list_for_graph[typ][i - 1]]
                    if now_node != pre_node:
                        G_in[typ].add_edge(pre_node, now_node)
                        G_out[typ].add_edge(now_node, pre_node)
                    else:
                        pass

        return G_in, G_out, train_data, test_data


    def build_adj(self, G, M):
        # M: number of adj per node
        N = args.nb_nodes
        # adj shape: [N, M]
        adj = [None] * N
        adj[0] = [0] * M

        w = [None] * N
        w[0] = [0] * M

        rdm = np.random.RandomState(555)
        pbar = tqdm(total=N - 1, desc='building adj')
        for node in range(1, N):
            pbar.update(1)
            adj_list = G.get_adj(node)
            if len(adj_list) > M:
                adj_list = rdm.choice(adj_list, size=M, replace=False).tolist()
            mask = [0] * (M - len(adj_list))
            adj_list = adj_list[:] + mask
            adj[node] = adj_list
            w_list = [G.edge_cnt.get((node, x), 0) for x in adj_list]
            w[node] = w_list
        pbar.close()
        return [adj, w]

class Data:
    def __init__(self):
        self.dp = DataProcess(args.ds, args.adj_length, args.seq_length)

        self.adjs = self.dp.adjs
        self.vid2node = self.dp.vid2node

        self.load_data()
        self.status = 'train'

    def load_data(self):
        self.data = self.dp.train_data + self.dp.test_data
        nb_train = len(self.dp.train_data)
        nb_non_train = len(self.dp.test_data)
        nb_vali = nb_non_train // 3
        nb_test = nb_non_train - nb_vali

        nb_data = len(self.data)
        assert nb_data > 0
        args.update(nb_data=nb_data, nb_train=nb_train, nb_vali=nb_vali, nb_test=nb_test)

    def pad_seq(self, node_list):
        L = args.seq_length
        if len(node_list) < L:
            node_list = node_list + [0] * (L - len(node_list))
        return node_list

    def sample_neg(self, pos, rdm):
        neg = set()
        while len(neg) < args.num_neg:
            n = rdm.randint(args.nb_nodes)
            if n != 0 and n != pos and n not in neg:
                neg.add(n)
        neg = sorted(neg)
        return neg

    def get_data_by_idx(self, idx, rdm):
        share_history, click_history, pos = self.data[idx]
        pos = self.vid2node[pos]

        share_seq = [self.vid2node[vid] for vid in share_history]
        click_seq = [self.vid2node[vid] for vid in click_history]

        share_list = self.pad_seq(share_seq)
        click_list = self.pad_seq(click_seq)

        ret = [share_list, click_list, pos]

        if self.status == 'train':
            neg = self.sample_neg(pos, rdm)
            ret.append(neg)
        return ret

    def get_batch_by_idxs(self, idxs, rdm=None):
        data = None
        for idx in idxs:
            d = self.get_data_by_idx(idx, rdm)
            n = len(d)
            if data is None:
                data = [[] for _ in range(n)]
            for i in range(n):
                data[i].append(d[i])

        # data: [0-seq, 1-typ, 2-len, 3-nxt, 4-label]
        batch = [np.array(d) for d in data]
        return batch

    def gen_train_batch_for_train(self, batch_size):
        rdm = np.random.RandomState(333)
        while True:
            idxs = list(range(args.nb_train))
            rdm.shuffle(idxs)
            for i in range(0, args.nb_train, batch_size):
                batch = self.get_batch_by_idxs(idxs[i: i + batch_size], rdm)
                yield batch

    def get_data_idxs(self, name):
        if name == 'train':
            return 0, args.nb_train
        if name == 'vali':
            return args.nb_train, args.nb_train + args.nb_vali
        if name == 'test':
            return args.nb_train + args.nb_vali, args.nb_data

    def gen_metric_batch(self, name, batch_size):
        self.status = 'metric'
        begin_idx, end_idx = self.get_data_idxs(name)
        yield from self.gen_data_batch(begin_idx, end_idx, batch_size)
        self.status = 'train'

    def gen_all_batch(self, batch_size):
        begin_idx = 0
        end_idx = args.nb_data
        yield from self.gen_data_batch(begin_idx, end_idx, batch_size)

    def gen_data_batch(self, begin_idx, end_idx, batch_size):
        for i in range(begin_idx, end_idx, batch_size):
            a, b = i, min(end_idx, i + batch_size)
            batch = self.get_batch_by_idxs(range(a, b))
            yield batch


    def metric(self, pred_list, true_vid):
        pred_list = np.array(pred_list)
        true_vid = np.expand_dims(np.array(true_vid), -1)
        print(pred_list.shape)
        print(true_vid.shape)

        k = 100
        acc_ar = (pred_list == true_vid)[:, :k]  # [BS, K]
        acc = acc_ar.sum(-1)

        rank = np.argmax(acc_ar[:, :k], -1) + 1
        mrr = (acc / rank).mean()
        ndcg = (acc / np.log2(rank + 1)).mean()

        acc = acc.mean()
        # print(acc_ar)
        # print(mrr)
        # input()
        acc *= 100
        mrr *= 100
        ndcg *= 100
        ret = acc
        return ret, '{:.3f},{:.4f},{:.4f}'.format(acc, mrr, ndcg)


def main():
    pass


if __name__ == '__main__':
    main()

## Train

The model training process is defined using the following codes.

In [4]:
import time

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
import utils
from utils import args, tqdm


class Train:
    def __init__(self, Model, data):
        self.data = data
        self.build_model(Model)
        self.has_train = False

    def build_model(self, Model):
        self.model = Model(self.data)

    def train(self):
        brk = 0
        best_vali = 0
        data_generator = self.data.gen_train_batch_for_train(args.batch_size)
        for ep in range(args.epochs):
            pbar = tqdm(total=args.nb_vali_step, desc='training', leave=False)
            loss = []
            t0 = time.time()
            for _ in range(args.nb_vali_step):
                data = next(data_generator)
                _loss = self.model.fit(data)

                loss.append(_loss)
                pbar.update(1)
            pbar.close()
            train_time = time.time() - t0

            vali_v, vali_str = self.metric('vali')
            if vali_v > best_vali:
                brk = 0
                best_vali = vali_v
                self.model.save()
            else:
                brk += 1
            red = (brk == 0)

            msg = f'#{ep + 1}/{args.epochs} loss: {np.mean(loss):.5f}, brk: {brk}, vali: {vali_str}'
            if args.show_test and args.nb_test > 0:
                _, test_str = self.metric('test')
                msg = f'{msg}, test: {test_str}'
            vali_time = time.time() - t0 - train_time
            msg = f'{msg}, time: {train_time:.0f}s,{vali_time:.0f}s'

            args.log.log(msg, red=red)

            if ep < 60:
                brk = 0
            if brk >= args.early_stopping:
                break
            self.has_train = True

    def final_test(self):
        self.model.restore()
        _, ret = self.metric('test')
        return ret


    def metric(self, name):
        data_gen = self.data.gen_metric_batch(name, batch_size=256)
        pred_list, true_vid = self.topk(data_gen)

        pred_list = np.array(pred_list)
        true_vid = np.expand_dims(np.array(true_vid), -1)

        k = 100
        acc_ar = (pred_list == true_vid)[:, :k]  # [BS, K]
        acc = acc_ar.sum(-1)

        rank = np.argmax(acc_ar[:, :k], -1) + 1
        mrr = (acc / rank).mean()
        ndcg = (acc / np.log2(rank + 1)).mean()

        acc = acc.mean()
        # print(acc_ar)
        # print(mrr)
        # input()
        acc *= 100
        mrr *= 100
        ndcg *= 100
        ret = acc
        # ret = acc + mrr * 10 + ndcg * 5
        # ret = acc + mrr + ndcg
        # return ret, 'HR%:{:.3f},MRR%:{:.4f},NDCG%:{:.4f}'.format(acc, mrr, ndcg)
        return ret, '{:.3f},{:.4f},{:.4f}'.format(acc, mrr, ndcg)

    def topk(self, data_gen):
        pred_list = []
        true_vid = []
        cnt = 0
        pbar = tqdm(desc='predicting...', leave=False)
        for data in data_gen:
            v, i = self.model.topk(data)
            pred_list.extend(i.tolist())
            true_vid.extend(data[2])
            pbar.update(1)
            cnt += 1
            if args.run_test and cnt > 3:
                break
        pbar.close()
        return pred_list, true_vid


def main():
    print('hello world, Train.py')


if __name__ == '__main__':
    main()

hello world, Train.py


## Main

In this part, we start the training process by determining the adjustable hyperparameter values of the model, for example, adjusting the depth of the neural network of the graph used or the size of the mini-Batch.

In [5]:
import argparse
import time

import numpy as np
import os
import random
import sys

import Train
# import dataset_online as dataset
import dataset
import models
import util
from util import args

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


def parse_args():
    parser = argparse.ArgumentParser()
    args, unknown = parser.parse_known_args()
    parser.add_argument('-show_test', action='store_true')
    parser.add_argument('-run_tb', action='store_true')
    parser.add_argument('-run_test', action='store_true')
    parser.add_argument('-epochs', type=int, default=100)
    parser.add_argument('-es', '--early_stopping', type=int, default=30)
    parser.add_argument('-valistep', '--nb_vali_step', type=int, default=500)
    parser.add_argument('-bs', '--batch_size', type=int, default=64)
    parser.add_argument('-seed', type=int, default=123456)
    parser.add_argument('-gpu', type=str, default='0')
    parser.add_argument('-ds', type=str, default='yc')
    parser.add_argument('-verbose', type=int, default=1)
    parser.add_argument('-msg', type=str, default='')
    # parser.add_argument('-restore_model', type = str, default = '')
    parser.add_argument('-model', type=str, default='GNN')

    parser.add_argument('-k', '--dim_k', type=int, default=64)
    parser.add_argument('-lr', type=float, default=1e-3)
    parser.add_argument('-l2_all', type=float, default=0)

    parser.add_argument('-seq_length', type=int, default=20)
    parser.add_argument('-adj_length', type=int, default=22)
    parser.add_argument('-num_neg', type=int, default=20)

    parser.add_argument('-gnnd', type=int, default=1)

    #a = parser.parse_args().__dict__
    arg, unknown = parser.parse_known_args()
    a = arg.__dict__
    return a


def main(**main_args):
    begin_time = time.time()

    # init args
    args.update(**main_args)
    command_line_args = parse_args()
    args.setdefault(**command_line_args)

    args.update(run_on_yard=True)

    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)


    # get Model, set model default args
    Model = vars(models)[args.model]
    args.setdefault(**Model.args.vars())
    #args.setdefault(**GNN.args.vars())

    if args.run_test:
        args.update(epochs=2, nb_vali_step=2, max_data_line=100)

    print(args)


    # get data
    random.seed(seed)
    np.random.seed(args.seed)
    data = Data()
    min_epochs = args.nb_train / (args.batch_size * args.nb_vali_step)
    if min_epochs < 1.0:
        args.update(nb_vali_step=int(np.ceil(args.nb_train / args.batch_size)))
        print(args)
        min_epochs = args.nb_train / (args.batch_size * args.nb_vali_step)
    args.update(min_epochs=int(np.ceil(min_epochs)))
    # args.setdefault())

    # run_name: time-x-Modes-ds
    time_str = get_time_str()
    model_name = Model.__name__
    #model_name = 'GNN'
    run_name = f'{time_str}-{model_name}-{args.ds}'
    if args.msg:
        run_name = f'{run_name}-{args.msg}'
    if args.run_test:
        run_name = f'{run_name}-test'

    args.update(run_name=run_name)
    #T =Train(Model, data)
    T =Train.Train(GNN, data)

    log_fn = f'{log_dir}/{run_name}.log'
    begin_time_str = get_time_str()
    print(begin_time_str, log_fn, '----- start!, pid:', os.getpid())
    args.update(pid=os.getpid())
    log = Logger(fn=log_fn, verbose=args.verbose)
    args.update(log=log)
    args.log.log(f'argv: {" ".join(sys.argv)}')
    args.log.log(f'log_fn: {log_fn}')
    args.log.log(f'args: {args.prt_json()}')
    args.log.log(f'Model: {model_name}')
    args.log.log(f'begin time: {begin_time_str}')

    try:
        T.train()
    except KeyboardInterrupt as e:
        if not T.has_train:
            raise e
    test_str = T.final_test()

    args.log.log(f'\ntest: {test_str}\n', red=True)


    args.log.log(log_fn)
    dt = time.time() - begin_time
    end_time_str = get_time_str()
    args.log.log(f'end time: {end_time_str}, dt: {dt / 3600:.2f}h')
    print(end_time_str, log_fn, f'##### over, time: {dt / 3600:.2f}h')


if __name__ == '__main__':
    print(os.getcwd())
    main()


C:\Users\Sharif\Desktop\new_project\MGNN-SPred-master(1)\MGNN-SPred-master
adj_length: 22
batch_size: 64
dim_k: 64
ds: yc
early_stopping: 30
epochs: 100
gnnd: 1
gpu: 0
l2_all: 0
lr: 0.001
model: GNN
msg: 
nb_vali_step: 500
num_neg: 20
run_on_yard: True
run_tb: False
run_test: False
seed: 123456
seq_length: 20
show_test: False
verbose: 1


building adj: 100%|##############################| 52739/52739 [00:01<00:00, 40945.99it/s]
<ipython-input-2-96b204ebe9d4>:173: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  alpha = tf.layers.dense(tf.concat([a, b], -1),
<ipython-input-2-96b204ebe9d4>:222: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  seq_emb = tf.layers.dense(seq_emb,args.dim_k,name='dense_W',use_bias=False)


seq_emb (?, 64)
embedding_matrix (52740, 64)
graph <tf.Variable 'Graph/emb_w:0' shape=(52740, 64) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge_share_and_click_seq/gateW/kernel:0' shape=(128, 1) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge_share_and_click_seq/gateW/bias:0' shape=(1,) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge_share_and_click_seqn/gateW/kernel:0' shape=(128, 1) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge_share_and_click_seqn/gateW/bias:0' shape=(1,) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge/gateW/kernel:0' shape=(128, 1) dtype=float32_ref>
network <tf.Variable 'Network/merge_seq/user_emb/merge/gateW/bias:0' shape=(1,) dtype=float32_ref>
network <tf.Variable 'Network/dense_W/kernel:0' shape=(64, 64) dtype=float32_ref>
2023-05-17_12.49.56.1534 run_time/log/2023-05-17_12.49.50.4390-GNN-yc.log ----- start!, pid: 8780
argv: C:\

#1/100 loss: 0.02902, brk: 0, vali: 5.236,0.2344,1.0685, time: 59s,10s


#2/100 loss: 0.02265, brk: 0, vali: 11.204,0.6959,2.4839, time: 60s,9s


...

#97/100 loss: 0.00902, brk: 3, vali: 26.105,3.0892,7.3443, time: 60s,8s


#98/100 loss: 0.00841, brk: 4, vali: 25.735,3.2427,7.3862, time: 61s,8s


#99/100 loss: 0.00850, brk: 5, vali: 25.951,2.9614,7.2157, time: 60s,8s


#100/100 loss: 0.00881, brk: 6, vali: 26.652,3.1499,7.4995, time: 60s,8s
INFO:tensorflow:Restoring parameters from run_time/save/2023-05-17_12.49.50.4390-GNN-yc/model.ckpt



test: 26.685,3.4416,7.7391

run_time/log/2023-05-17_12.49.50.4390-GNN-yc.log
end time: 2023-05-17_14.41.31.2552, dt: 1.90h
2023-05-17_14.41.31.2552 run_time/log/2023-05-17_12.49.50.4390-GNN-yc.log ##### over, time: 1.90h
